In [ ]:
import cv2
import numpy as np
import dlib
import pyautogui
import speech_recognition as sr
import threading
import keyboard

# Constantes
SHAPE_PREDICTOR_FILE = "C:\\Users\\Utilisateur\\Downloads\\gaze_controlled_keyboard_p10\\shape_predictor_68_face_landmarks.dat"

# Fonction pour calculer le point médian entre deux points
def midpoint(p1, p2):
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y) / 2)

# Fonction pour la détection du visage en utilisant dlib
def detect_face(frame, predictor):
    detector = dlib.get_frontal_face_detector()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        left_eye_mid = midpoint(landmarks.part(42), landmarks.part(45))
        right_eye_mid = midpoint(landmarks.part(36), landmarks.part(39))

        # Dessiner les points pour les yeux (pour le débogage)
        cv2.circle(frame, left_eye_mid, 5, (0, 0, 255), -1)
        cv2.circle(frame, right_eye_mid, 5, (0, 0, 255), -1)

        return right_eye_mid, left_eye_mid  # Inverser left et right

    return None, None

# Fonction principale de l'interaction avec la souris basée sur les mouvements des yeux
def eye_mouse_interaction(quit_program):
    # Initialisation de la capture vidéo
    cap = cv2.VideoCapture(0)  # Ajustez l'index si nécessaire

    # Vérifiez si la capture vidéo est réussie
    if not cap.isOpened():
        print("Erreur : Impossible d'ouvrir la caméra.")
        quit_program.set()
        return
    else:
        print("Caméra ouverte avec succès.")

    # Initialisation du détecteur de visage et des points de repère faciaux
    predictor = dlib.shape_predictor(SHAPE_PREDICTOR_FILE)

    prev_right_eye_mid = (0, 0)

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Erreur : Impossible de lire la trame de la caméra.")
            break

        # Détection du visage et des yeux
        right_eye, left_eye = detect_face(frame, predictor)  # Inverser left et right

        # Logique d'interaction avec la souris
        if left_eye is not None and right_eye is not None:
            # Déplacement de la souris en fonction du mouvement des yeux
            if prev_right_eye_mid != (0, 0):
                delta_x = -right_eye[0] + prev_right_eye_mid[0]  # Inverser left et right
                delta_y = left_eye[1] - prev_right_eye_mid[1]  # Inverser left et right
                pyautogui.moveRel(delta_x, delta_y)

            prev_right_eye_mid = right_eye

        # Affichage de la vidéo capturée
        cv2.imshow("Camera", frame)

        key = cv2.waitKey(1)
        if key == 27:  # Touche 'Esc'
            break

        if quit_program.is_set():
            break

    cap.release()
    cv2.destroyAllWindows()

# Fonction pour activer le clic de la souris en réponse à une commande vocale ou à la pression de la touche "Echap"
def activate_mouse_click(quit_program):
    recognizer = sr.Recognizer()

    while True:
        with sr.Microphone() as source:
            print("Dites quelque chose...")
            audio = recognizer.listen(source)

        try:
            # Utiliser l'API Google pour la reconnaissance vocale
            text = recognizer.recognize_google(audio, language="fr-FR")
            print(f"Vous avez dit: {text}")

            # Vérifier si le texte est "ok" pour activer le clic de la souris
            if "ok" in text.lower():
                # Simuler le clic de la souris sans déplacer la position actuelle
                pyautogui.click(interval=0.1)  # Réduit le temps d'attente du clic à 0.1 seconde
                print("Clic de la souris activé.")
                
            # Vérifier si le texte est "arrêt" pour arrêter le programme
            if "arrêt" in text.lower():
                quit_program.set()
                break

        except sr.UnknownValueError:
            print("Impossible de comprendre l'audio.")
        except sr.RequestError as e:
            print(f"Erreur de requête pour la reconnaissance vocale; {e}")

        # Si la touche 'Echap' est pressée, arrêtez le programme
        if keyboard.is_pressed('esc'):
            quit_program.set()
            break

# Fonction principale pour exécuter les deux fonctions dans des threads séparés
def main():
    quit_program = threading.Event()

    thread_eye_mouse_interaction = threading.Thread(target=eye_mouse_interaction, args=(quit_program,))
    thread_activate_mouse_click = threading.Thread(target=activate_mouse_click, args=(quit_program,))

    thread_eye_mouse_interaction.start()
    thread_activate_mouse_click.start()

    thread_eye_mouse_interaction.join()
    thread_activate_mouse_click.join()

if __name__ == "__main__":
    main()
